In [1]:
%pip install faker
%pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 480.7 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from faker import Faker
from faker.providers import DynamicProvider
import pandas as pd
import random
import numpy as np
age = DynamicProvider(
     provider_name="age",
     elements=[i for i in range(21,70)],
)
gender = DynamicProvider(
     provider_name="gender",
     elements=["Male","Female"],
)
income=DynamicProvider(
    provider_name="income",
    elements=[i for i in range(3000,15000)],

)
education = DynamicProvider(
     provider_name="education",
     elements=["Primary","Secondary","Tertiary","Postgrad"],
)
campaign_type= DynamicProvider(
    provider_name="campaign_type",
    elements=["Facebook","Instagram","X"],
)
def random_campaign_types():
    x= random.sample(["Facebook", "Instagram", "Twitter"], random.randint(1, 3))
    num=random.randint(0,1)
    if num ==1:
        x.append("Email")
    return x
def random_products():
    x = random.sample(["W","X","Y","Z"], random.randint(0, 4))
    return x
duration = DynamicProvider(
     provider_name="duration",
     elements=[15,30,45,60],
)
product_type= DynamicProvider(
    provider_name="product_type",
    elements=["A","B","C","D","E"],
)
prev_campaign=DynamicProvider(
    provider_name="prev_campaign",
    elements=["Yes","No"],
)
outcome=DynamicProvider(
    provider_name="outcome",
    elements=["Yes","No"],
)
views=DynamicProvider(
    provider_name="views",
    elements=[i for i in range(1,10)],
)

def fake_loyalty(a,b):
    return random.randint(a,b)
def fake_balance(a,b):
    return random.randint(a,b)
def fake_adspend(a,b):
    return random.uniform(a,b)

product_cost={"A":100,"B":200,"C":300,"D":400,"E":500}
marketing_cost={"Facebook":10000,"X":18000,"Instagram":20000}
fake = Faker()
lst=[["customer id","age","gender","income/month","account balance","loyalty score","education level","products owned","type of campaign(s)","duration of campaign(s) (days)","total ad spend","product type advertised"
      , "prev campaign success","product cost","has customer bought product"],]
# then add new provider to faker instance
fake.add_provider(age)
fake.add_provider(gender)
fake.add_provider(income)
fake.add_provider(education)
fake.add_provider(campaign_type)
fake.add_provider(duration)
fake.add_provider(product_type)
fake.add_provider(prev_campaign)
fake.add_provider(outcome)
fake.add_provider(views)

for i in range(20000):

    l=[i+1,fake.age(),fake.gender(),fake.income(),fake_balance(100,10000),fake_loyalty(0,1000),fake.education(),random_products(),random_campaign_types(),15,fake_adspend(200,900)
      , fake.product_type(),fake.prev_campaign()]
    prod_cost=product_cost[l[-2]]
    l.append(prod_cost)
    l.append(fake.outcome())
    lst.append(l)

df=pd.DataFrame(lst[1:],columns=lst[0])
df["Facebook ad [clicks, impressions, CTR(Click thru rate)]"]=None
df["Twitter ad [clicks, impressions, CTR(Click thru rate)]"]=None
df["Instagram ad [clicks, impressions, CTR(Click thru rate)]"]=None
df["Total ad [clicks, impressions, CTR(Click thru rate)]"]=None
df["Email [Opens,clicks]"]=None

def ad_generator(row):
    platforms=row["type of campaign(s)"]
    total_clicks=0
    total_impressions=0
    if "Twitter" in platforms:
        twitter_clicks=int(random.randint(1,4))
        total_clicks +=twitter_clicks

        twitter_impressions=random.randint(4* twitter_clicks, 20* twitter_clicks)
        total_impressions+=twitter_impressions

        twitter_ctr=round(twitter_clicks/twitter_impressions,5)
        row["Twitter ad [clicks, impressions, CTR(Click thru rate)]"]= [twitter_clicks,twitter_impressions,twitter_ctr]

    if "Facebook" in platforms:
        facebook_clicks=int(random.randint(1,4))
        total_clicks += facebook_clicks

        facebook_impressions=random.randint(4* facebook_clicks, 20* facebook_clicks)
        total_impressions+=facebook_impressions

        facebook_ctr=round(facebook_clicks/facebook_impressions,5)
        row["Facebook ad [clicks, impressions, CTR(Click thru rate)]"] = [facebook_clicks,facebook_impressions,facebook_ctr]

    if "Instagram" in platforms:
        instagram_clicks=int(random.randint(1,4))
        total_clicks+=instagram_clicks

        instagram_impressions=random.randint(4* instagram_clicks, 20* instagram_clicks)
        total_impressions+=instagram_impressions

        instagram_ctr=round(instagram_clicks/instagram_impressions,5)
        row["Instagram ad [clicks, impressions, CTR(Click thru rate)]"] = [instagram_clicks,instagram_impressions,instagram_ctr]

    if "Email" in platforms:
        email_opens=random.randint(1,9)
        email_clicks=random.randint(0,email_opens)
        row["Email [Opens,clicks]"]=[email_opens,email_clicks]


    avg_ctr=round(total_clicks/total_impressions,5)
    row["Total ad [clicks, impressions, CTR(Click thru rate)]"]=[total_clicks,total_impressions,avg_ctr]
    return row
df2=df.apply(ad_generator,axis=1)

def generate_transactions(row):
    k=random.randint(0,6)
    dates=sorted(list(random.sample(range(-60,-1),k)))
    l=[]
    for date in dates:
        transaction_type=str((np.random.choice(["Withdrawal","Deposit"],size=1))[0])
        if transaction_type=="Withdrawal":
            amt=round(random.uniform(10,5000),2)
        else:
            amt=round(random.uniform(100,10000),2)
        l.append([date,transaction_type,amt])
    row["Transaction history"]=l
    return row

df3=df2.apply(generate_transactions,axis=1)


df3["has customer bought product"]=df3.pop("has customer bought product")
df3["has customer churned"]=np.random.choice(["Yes","No"],size=len(df3))



In [3]:
df3.columns

Index(['customer id', 'age', 'gender', 'income/month', 'account balance',
       'loyalty score', 'education level', 'products owned',
       'type of campaign(s)', 'duration of campaign(s) (days)',
       'total ad spend', 'product type advertised', 'prev campaign success',
       'product cost',
       'Facebook ad [clicks, impressions, CTR(Click thru rate)]',
       'Twitter ad [clicks, impressions, CTR(Click thru rate)]',
       'Instagram ad [clicks, impressions, CTR(Click thru rate)]',
       'Total ad [clicks, impressions, CTR(Click thru rate)]',
       'Email [Opens,clicks]', 'Transaction history',
       'has customer bought product', 'has customer churned'],
      dtype='object')

# Customer attributes(for segmentation):
- age
- gender
- income
- account balance
- education level

#Attributes for recommendation system
- products already owned by customer
- product advertised during campaign
- transaction history
-did customer buy advertised product (Yes/No)

#KPI for campaign success
- Click through rate(CTR)
- Conversion rate for each campaign(calculated in analysis below)
- Custom ROI metrics (ad spend relative to product cost, CAC, etc.)

#Customer campaign engagement metrics
- (Online channels) Facebook/twitter/instagram ad [clicks/impressions(total no. of ads shown)/CTR]
- (Offline channel) Email [clicks/opens(no. of times customer opens link in email)]

#Customer churn metrics
- Transaction history [date, transaction type, transaction amount] (only up to past 60 days)
- loyalty score (based on amount of years in bank + transaction frequency)

#Labels
- did customer buy advertised product
- did customer churn


In [4]:
df3

,customer id,age,gender,income/month,account balance,loyalty score,education level,products owned,type of campaign(s),duration of campaign(s) (days),...,prev campaign success,product cost,"Facebook ad [clicks, impressions, CTR(Click thru rate)]","Twitter ad [clicks, impressions, CTR(Click thru rate)]","Instagram ad [clicks, impressions, CTR(Click thru rate)]","Total ad [clicks, impressions, CTR(Click thru rate)]","Email [Opens,clicks]",Transaction history,has customer bought product,has customer churned
0,1,67,Male,12223,7018,721,Secondary,[X],"[Instagram, Twitter]",15,...,No,400,None,"[2, 17, 0.11765]","[2, 38, 0.05263]","[4, 55, 0.07273]",None,"[[-55, Withdrawal, 3929.99], [-28, Withdrawal,...",No,No
1,2,26,Female,11316,927,700,Postgrad,[Y],"[Twitter, Instagram, Email]",15,...,Yes,300,None,"[3, 46, 0.06522]","[2, 38, 0.05263]","[5, 84, 0.05952]","[7, 2]",[],Yes,No
2,3,67,Male,4493,8729,911,Tertiary,[],"[Facebook, Instagram, Twitter]",15,...,No,500,"[1, 15, 0.06667]","[3, 35, 0.08571]","[3, 44, 0.06818]","[7, 94, 0.07447]",None,"[[-45, Deposit, 2801.75], [-41, Deposit, 8932....",No,No
3,4,52,Female,14545,1556,291,Primary,[],"[Instagram, Email]",15,...,Yes,200,None,None,"[3, 41, 0.07317]","[3, 41, 0.07317]","[8, 0]","[[-45, Withdrawal, 4267.88], [-23, Deposit, 85...",Yes,Yes
4,5,45,Female,12496,6967,487,Postgrad,"[Y, W, Z, X]","[Instagram, Email]",15,...,No,200,None,None,"[3, 51, 0.05882]","[3, 51, 0.05882]","[9, 3]","[[-57, Withdrawal, 1938.57], [-55, Deposit, 85...",Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,29,Male,10478,2357,892,Primary,[],"[Facebook, Instagram]",15,...,Yes,400,"[3, 30, 0.1]",None,"[1, 10, 0.1]","[4, 40, 0.1]",None,"[[-50, Deposit, 1205.68], [-38, Deposit, 6380....",Yes,Yes
19996,19997,29,Female,9472,9955,419,Tertiary,"[Y, Z, W]","[Twitter, Facebook]",15,...,Yes,500,"[1, 11, 0.09091]","[1, 7, 0.14286]",None,"[2, 18, 0.11111]",None,"[[-58, Deposit, 9149.08], [-46, Withdrawal, 23...",No,Yes
19997,19998,63,Female,5644,9945,760,Tertiary,"[Z, X, Y, W]","[Twitter, Email]",15,...,No,400,None,"[2, 8, 0.25]",None,"[2, 8, 0.25]","[1, 0]","[[-39, Withdrawal, 78.52]]",Yes,Yes
19998,19999,61,Male,8952,2279,875,Primary,"[X, W, Y, Z]","[Twitter, Facebook, Instagram, Email]",15,...,No,500,"[1, 12, 0.08333]","[2, 36, 0.05556]","[4, 50, 0.08]","[7, 98, 0.07143]","[7, 2]","[[-59, Withdrawal, 4656.24], [-41, Withdrawal,...",Yes,No


In [11]:
df3.to_csv("my_dataset.csv", index=False)  # Set index=True if you want to keep the index